## Tarea 7 del curso CM-072

* Nombre y apellidos: Bitzer Arotoma Bacilio
* Fecha de presentación: 17 de octubre.



LendingClub es una compañía de préstamos *peer-to-peer* que conecta directamente a los prestatarios con potenciales prestamistas/inversionistas.

Construirás un modelo de clasificación para predecir si un préstamo realizado a través del LendingClub tiene probabilidad de no ser pagado.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

loans = pd.read_csv("lending-club-data.csv", low_memory=False)

1 . Carga en una variable de nombre `todo_columnas` el nombre de todas las columnas del conjunto de datos.

In [2]:
# Solucion
todo_columnas = loans.columns
todo_columnas

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans',
       'emp_length_num', 'grade_num', 'sub_gra

2 . La columna que contiene la información que queremos predecir se llama `malos_prestamos`. En esta columna, el valor 1 significa un préstamo riesgoso (malo), mientras que 0 significa un préstamos seguro.

Para hacer el trabajo más intuitivo, crea una nueva columna `prestamos_seguros` con el siguiente valor:

* +1 si es un préstamo seguro
* -1 si es un préstamos riesgoso (malo)

In [3]:
# Solucion
loans['prestamos_seguros'] = loans.bad_loans.apply(lambda x : +1 if x==0 else -1)

3 . Calcula la distribución en porcentaje de préstamos malos y préstamos buenos (debe sumar 100%).

In [4]:
# Tu solución
loans.prestamos_seguros.value_counts(normalize=True)

/home/bitzer/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


 1    0.811185
-1    0.188815
Name: prestamos_seguros, dtype: float64

4 . Una manera de abordar conjuntos de datos desbalanceados es con un submuestreo  de la clase más grande hasta que la distribución de clases sea mitad y mitad. Vamos a realizar un submuestreo de los préstamos buenos para balancear nuestro conjunto de datos. Ello significa que vamos a descartar muchas observaciones. 

* Pon en una variable `prestamos_arriesgado` todos y solo los préstamos malos.
* Pon en una variable `prestamos_seguros` una muestra aleatoria de préstamos buenos **del mismo tamaño** que la cantidad de préstamos malos. (Usa [pandas.DataFrame.sample](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html) con el atributo `random_state=0`)
* Junta en una nueva variable `prestamos_balanceados`, los dos grupos anteriores: `prestamos_arriesgados` y `prestamos_seguros`.

In [5]:
# Tus soluciones
prestamos_arrriesgado = loans[loans.prestamos_seguros==-1]
prestamos_seguros = loans[loans.prestamos_seguros==1].sample(len(prestamos_arrriesgado),random_state=0)
prestamos_balanceados = prestamos_arrriesgado.append(prestamos_seguros)
prestamos_balanceados.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,prestamos_seguros
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1,-1
6,1071795,1306957,5600,5600,5600,60 months,21.28,152.39,F,F2,...,1.0,1.0,1.0,0,4.57170,20161201T000000,1,1,1,-1
7,1071570,1306721,5375,5375,5350,60 months,12.69,121.45,B,B5,...,1.0,1.0,1.0,1,9.71600,20161201T000000,1,1,1,-1
10,1064687,1298717,9000,9000,9000,36 months,13.49,305.38,C,C1,...,1.0,1.0,1.0,1,12.21520,20141201T000000,1,1,1,-1
12,1069057,1303503,10000,10000,10000,36 months,10.65,325.74,B,B2,...,1.0,1.0,1.0,0,3.90888,20141201T000000,1,1,1,-1


5 . Asigna a una variable `subconjunto_prestamos` sólo el siguiente subconjunto de características que son las que usaremos:

```python
caracteristica = ['grade',               # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]
```

Asimismo, asigna a una variable **`y`** los valores de la columna `prestamos_seguros`.

In [6]:
caracteristica = ['grade',               # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]
subconjunto_prestamos = prestamos_balanceados.loc[:,caracteristica]
y = prestamos_balanceados['prestamos_seguros']
y.shape

(46300,)

In [7]:
subconjunto_prestamos.shape

(46300, 12)

6 .  Usando [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) convierte las variables categóricas de `subconjunto_prestamos` en variables numéricas *one-hot*. Guarda el nuevo conjunto de datos en `X`.

In [8]:
# Tu solucion
X = pd.get_dummies(subconjunto_prestamos)
X.shape

(46300, 67)

7 .  Empleando `sklearn.model_selection.train_test_split` separa el conjunto de datos en un 90% para entrenamiento y validación (`X_entrenamiento_val`, `y_entrenamiento_val`), y 10% para pruebas (`X_prueba`, `y_prueba`).

Luego separa (`X_entrenamiento_val`, `y_entrenamiento_val`) en un 80% para entrenamiento (`X_entrenamiento`, `y_entrenamiento`) y 20% para validación (`X_val`, `y_val`).

In [9]:
# Tu solucion
from sklearn.model_selection import train_test_split
X_entrenamiento_val, X_prueba, y_entrenamiento_val, y_prueba = train_test_split( X, y, test_size=0.1) 
X_entrenamiento, X_val, y_entrenamiento, y_val = train_test_split( X_entrenamiento_val, y_entrenamiento_val, test_size=0.2)

8 . Entrena un modelo como Regresión Logística, Naive Bayes, KNN y un cuarto modelo de tu elección, con las siguientes indicaciones:

* Utilizar el uso apropiado de la normalización (Scaling) de datos si fuese necesario.
* El uso apropiado de una técnica para la selección de los mejores parámetros de cada modelo (p.ej. búsqueda grid o búsqueda aleatoria)
* Reporte para cada modelo la exactitud , precisión y exhaustividad, F1-Score  **en el conjunto de pruebas.** y muestra la matriz de confusión.
* Comenta tus resultados.

In [10]:
# Tus soluciones
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics 


# Regresión Logística 

In [11]:
logR = LogisticRegression()
logR.fit(X_entrenamiento_val,y_entrenamiento_val)

print("<<<<Regresion logistica>>>>>")
print("Training")
print("========")
y_predict = logR.predict(X_entrenamiento_val)
print("Exactitud : ",metrics.accuracy_score(y_entrenamiento_val,y_predict))
print("Precision : ",metrics.precision_score(y_entrenamiento_val,y_predict))
print("F1-score : ",metrics.f1_score(y_entrenamiento_val,y_predict))
print("Exhaustividad:", metrics.recall_score(y_entrenamiento_val, y_predict))
m_c = metrics.confusion_matrix(y_entrenamiento_val,y_predict)
print("Matriz de confusion:")
print(m_c)

print("\nTesting")
print("========")
y_predict = logR.predict(X_prueba)
print("Exactitud : ",metrics.accuracy_score(y_prueba,y_predict))
print("Precision : ",metrics.precision_score(y_prueba,y_predict))
print("F1-score : ",metrics.f1_score(y_prueba,y_predict))
print("Exhaustividad:", metrics.recall_score(y_prueba, y_predict))

m_c = metrics.confusion_matrix(y_prueba,y_predict)
print("Matriz de confusion:")
print(m_c)


<<<<Regresion logistica>>>>>
Training
Exactitud :  0.6461963042956563
Precision :  0.6472191669456268
F1-score :  0.6473894429695535
Exhaustividad: 0.6475598086124402
Matriz de confusion:
[[13393  7377]
 [ 7366 13534]]

Testing
Exactitud :  0.629805615550756
Precision :  0.6154177433247201
F1-score :  0.6251093613298339
Exhaustividad: 0.6351111111111111
Matriz de confusion:
[[1487  893]
 [ 821 1429]]


# Naive Bayes

In [12]:
#segundo modelo
gnb = MultinomialNB()
gnb.fit(X_entrenamiento_val,y_entrenamiento_val);
print("<<<<Naive bayes>>>>>")
print("Training")
print("========")
y_predict = gnb.predict(X_entrenamiento_val)
print("Exactitud : ",metrics.accuracy_score(y_entrenamiento_val,y_predict))
print("Precision : ",metrics.precision_score(y_entrenamiento_val,y_predict))
print("F1-score : ",metrics.f1_score(y_entrenamiento_val,y_predict))
print("Exhaustividad:", metrics.recall_score(y_entrenamiento_val, y_predict))
m_c = metrics.confusion_matrix(y_entrenamiento_val,y_predict)
print("Matriz de confusion:")
print(m_c)

print("\nTesting")
print("========")
y_predict = gnb.predict(X_prueba)
print("Exactitud : ",metrics.accuracy_score(y_prueba,y_predict))
print("Precision : ",metrics.precision_score(y_prueba,y_predict))
print("F1-score : ",metrics.f1_score(y_prueba,y_predict))
print("Exhaustividad:", metrics.recall_score(y_prueba, y_predict))
m_c = metrics.confusion_matrix(y_prueba,y_predict)
print("Matriz de confusion:")
print(m_c)


<<<<Naive bayes>>>>>
Training
Exactitud :  0.5876169906407488
Precision :  0.554885974240813
F1-score :  0.6861484511981297
Exhaustividad: 0.898755980861244
Matriz de confusion:
[[ 5702 15068]
 [ 2116 18784]]

Testing
Exactitud :  0.5730021598272138
Precision :  0.5362165030512073
F1-score :  0.6715401229440107
Exhaustividad: 0.8982222222222223
Matriz de confusion:
[[ 632 1748]
 [ 229 2021]]


In [13]:
#tercer modelo
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_entrenamiento_val, y_entrenamiento_val)
print("<<<<KNN>>>>>")
print("Training")
print("========")
y_predict = neigh.predict(X_entrenamiento_val)
print("Exactitud : ",metrics.accuracy_score(y_entrenamiento_val,y_predict))
print("Precision : ",metrics.precision_score(y_entrenamiento_val,y_predict))
print("F1-score : ",metrics.f1_score(y_entrenamiento_val,y_predict))
print("Exhaustividad:", metrics.recall_score(y_entrenamiento_val, y_predict))
m_c = metrics.confusion_matrix(y_entrenamiento_val,y_predict)
print("Matriz de confusion:")
print(m_c)
print("Dimension : ",len(y_predict))

print("\nTesting")
print("========")
y_predict = neigh.predict(X_prueba)
print("Exactitud : ",metrics.accuracy_score(y_prueba,y_predict))
print("Precision : ",metrics.precision_score(y_prueba,y_predict))
print("F1-score : ",metrics.f1_score(y_prueba,y_predict))
print("Exhaustividad:", metrics.recall_score(y_prueba, y_predict))
m_c = metrics.confusion_matrix(y_prueba,y_predict)
print("Matriz de confusion:")
print(m_c)
print(" analizado : ",len(y_predict))

print("\nValidación")
print("========")
y_predict = neigh.predict(X_val)
print("Exactitud : ",metrics.accuracy_score(y_val,y_predict))
print("Precision : ",metrics.precision_score(y_val,y_predict))
print("F1-score : ",metrics.f1_score(y_val,y_predict))
print("Exhaustividad:", metrics.recall_score(y_val, y_predict))
m_c = metrics.confusion_matrix(y_val,y_predict)
print("Matriz de confusion:")
print(m_c)
print("Datos analizados : ",len(y_predict))

<<<<KNN>>>>>
Training
Exactitud :  0.7768178545716342
Precision :  0.7698678578075563
F1-score :  0.780618984714097
Exhaustividad: 0.7916746411483253
Matriz de confusion:
[[15824  4946]
 [ 4354 16546]]
Feactures analizados :  41670

Testing
Exactitud :  0.5578833693304536
Precision :  0.541010101010101
F1-score :  0.5667724867724868
Exhaustividad: 0.5951111111111111
Matriz de confusion:
[[1244 1136]
 [ 911 1339]]
Feactures analizados :  4630

Validación
Exactitud :  0.779097672186225
Precision :  0.7721430240710446
F1-score :  0.7821043910521956
Exhaustividad: 0.7923261390887291
Matriz de confusion:
[[3189  975]
 [ 866 3304]]
Feactures analizados :  8334


In [14]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=4)
clf.fit(X_entrenamiento_val, y_entrenamiento_val)
print("<<<<Random Forest>>>>>")
print("Training")
print("========")
y_predict = clf.predict(X_entrenamiento_val)
print("Exactitud : ",metrics.accuracy_score(y_entrenamiento_val,y_predict))
print("Precision : ",metrics.precision_score(y_entrenamiento_val,y_predict))
print("F1-score : ",metrics.f1_score(y_entrenamiento_val,y_predict))
print("Exhaustividad:", metrics.recall_score(y_entrenamiento_val, y_predict))
m_c = metrics.confusion_matrix(y_entrenamiento_val,y_predict)
print("Matriz de confusion:")
print(m_c)

print("\nTesting")
print("========")
y_predict = clf.predict(X_prueba)
print("Exactitud : ",metrics.accuracy_score(y_prueba,y_predict))
print("Precision : ",metrics.precision_score(y_prueba,y_predict))
print("F1-score : ",metrics.f1_score(y_prueba,y_predict))
print("Exhaustividad:", metrics.recall_score(y_prueba, y_predict))
m_c = metrics.confusion_matrix(y_prueba,y_predict)
print("Matriz de confusion:")
print(m_c)

<<<<Random Forest>>>>>
Training
Exactitud :  0.6320614350851932
Precision :  0.6220731386477243
F1-score :  0.6492014826339633
Exhaustividad: 0.6788038277511962
Matriz de confusion:
[[12151  8619]
 [ 6713 14187]]

Testing
Exactitud :  0.616414686825054
Precision :  0.5936758893280633
F1-score :  0.6284518828451883
Exhaustividad: 0.6675555555555556
Matriz de confusion:
[[1352 1028]
 [ 748 1502]]


# Comentarios 
De los 4 modelos el que presenta la precisión mas baja es el Naive Bayes pero a sus vez es el que presenta mayor exhaustividad lo que significa que hay pocos falsos negativos a comparación de los otros modelos pero no significa que sea un buen modelo ya que comparando ambas metricas la precisión tiene mayor peso a comparación de la exhaustividad, por lo que no es un buen modelo. El mejor modelo que se aplico en este caso es el KNN pero este solo se da cuando tienes una gran cantidad de data para el testing, ya que como se observa con la data del testing se tiene unas bajas metricas a comparación de la data de validación que posee el doble de data para analizar y tiene mayor scoring.